In [13]:
%pylab inline
%install_ext https://raw.github.com/cjdrake/ipython-magic/master/gvmagic.py
%load_ext gvmagic

import tensorflow as tf
sess = tf.InteractiveSession()

/Users/keveman/nb/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


/Users/keveman/nb/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed gvmagic.py. To use it, type:
  %load_ext gvmagic


## Language Modeling Using TensorFlow

* Task : Given a sequence of words, predict the next word
  - Models the probability of sentences in a language
* Data available at http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

In [11]:
!head -n3 data/ptb_word/ptb.train.txt

 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 


* Exactly 10000 words including `<unk>` and `<eos>` (End of sentence)

* Read the data

In [37]:
data = open('data/ptb_word/ptb.train.txt').read().replace('\n', '<eos>').split()
data_as_set = set(data)
print('Number of words %d' % len(data_as_set))
word_to_id = {w: i for i, w in enumerate(data_as_set)}
id_to_word = {i: w for i, w in enumerate(data_as_set)}

Number of words 10000
